In [22]:
import sqlmlutils
import pytest


connection = sqlmlutils.ConnectionInfo(
    server="myServerName",
    database="myDatabaseName",
    uid="sa", 
    pwd="myPassword"
)

def testSP():
    import pandas as pd
    import numpy as np
    import pyodbc

    conn = pyodbc.connect(r"driver={SQL SERVER};server=myServerName;database=myDatabaseName;trusted_connection=YES;")
    query2 = "SELECT TOP (1000) [PRS_START_TIME],[PRS_END_TIME],[PRS_STATE],[PRS_BUSINESS_KEY],try_convert(float, RIGHT(PRS_Business_key, len(PRS_BUSINESS_KEY) - charindex(':', prs_business_key))) as Business_Key, try_convert(float, DATEDIFF(MILLISECOND, PRS_START_TIME,PRS_END_TIME)) as Process_Time FROM [ORCHESTRA].[dbo].[ORC_PROCESS_STATE];"
    SQLquery = pd.read_sql_query(query2, conn)


    df = pd.DataFrame(SQLquery, columns=['PRS_START_TIME','PRS_END_TIME','PRS_STATE','Business_Key', 'Process_Time'])

    from sklearn.model_selection import train_test_split

    y = df['PRS_STATE']
    X = df[['Business_Key', 'Process_Time']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

    from sklearn.tree import DecisionTreeClassifier

    dtree = DecisionTreeClassifier()
    dtree.fit(X_train,y_train)

    predictions = dtree.predict(X_test)
    print(predictions)
    
sqlpy = sqlmlutils.SQLPythonExecutor(connection)
sp_name = "pyTestFunctionToStoredProc"

sqlpy.drop_sproc(sp_name)
sqlpy.create_sproc_from_function(sp_name, testSP)

True